In [1]:
# Parameters in this cell can be overriden using papermill

# path to directory contaning output from the extract_test_outputs method in analyze.py
ANALYSIS_DIR="."

# If no cached 'meshes.gz' file exists when loading data, setting
# DERIVE_MESHES to True causes the mesh state to be derived from
# trace events. This will add several minutes to the load time,
# but the results will be cached to disk.
DERIVE_MESHES=False

# When DERIVE_MESHES is true, you can change the sample frequency
# by setting MESH_SAMPLE_FREQ. This controls the size of the time window
# for each mesh "snapshot".
MESH_SAMPLE_FREQ='5s'

In [2]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import toml
import ipywidgets as widgets
from pprint import pprint
import pathlib
import seaborn as sns
from durations import Duration

import notebook_helper
from notebook_helper import no_scores_message, load_pandas, archive_figures, p25, p50, p75, p95, p99

# render charts in a larger, zoomable style
%matplotlib notebook

# turn off autosaving for the notebook
%autosave 0

# load data
print('loading test data from ' + ANALYSIS_DIR)

tables = load_pandas(ANALYSIS_DIR, derive_meshes_if_missing=DERIVE_MESHES, mesh_sample_freq=MESH_SAMPLE_FREQ)
scores = tables['scores']
scores = scores.dropna()
print("Normal scores\n", scores)
# copy the data
scores_striped = scores.copy()
scores_striped = scores_striped.drop('IPColocationFactor', axis=1)
scores_striped = scores_striped.drop('BehaviourPenalty', axis=1)
scores_striped = scores_striped.drop('blocks.InvalidMessageDeliveries', axis=1)
scores_striped = scores_striped.drop('AppSpecificScore', axis=1)
print("Striped scores\n", scores_striped)

score_values = scores_striped.iloc[: , -3:]

# apply normalization techniques
for column in score_values.columns:
    score_values[column] = (score_values[column] \
    - score_values[column].min()) / (score_values[column].max() \
    - score_values[column].min())
  
# view normalized data
print("Normalized scores\n", score_values)

metrics = tables['metrics']
cdf = tables['cdf']
pdf = tables['pdf']
peers = tables['peers']
meshes = tables.get('meshes', notebook_helper.empty_meshes_table())

# resample score index into 5s windows for plotting later
if not scores.empty:
    print('resampling peer scores')
    resample_interval = '5s'
    sampled = scores.resample(resample_interval)
    honest_sampled = scores.where(scores['honest']).resample(resample_interval)
    attacker_sampled = scores.where(scores['honest'] == False).resample(resample_interval)

honest_peers = peers.where(peers['honest']).dropna(how='all')
attack_peers = peers.where(~peers['honest']).dropna(how='all').dropna('columns')

t_warm = honest_peers['t_warm'].max()
t_run = honest_peers['t_run'].max()
t_cool = honest_peers['t_cool'].max()
t_complete = honest_peers['t_complete'].max()

# TODO: support multiple waves of attackers with different connect times
attack_connect_start = attack_peers['t_connect'].min()
attack_connect_end = attack_peers['t_connect'].max()

params_panel, test_params = notebook_helper.test_params_panel(ANALYSIS_DIR)

try:
    s = test_params['PEER_SCORE_PARAMS']['Topics']['blocks']['MeshMessageDeliveriesActivation']
    d = Duration(s)
    d = pd.Timedelta(d.to_seconds(), unit='seconds')
    mesh_activation_start = attack_connect_start + d
    mesh_activation_end = attack_connect_end + d
except BaseException as err:
    print('error determining mesh activation window: ', err)    


Autosave disabled
loading test data from .
Normal scores
                                observer  peer       score honest  \
timestamp                                                          
2022-11-04 00:25:06.011213244        39     5    0.000000   True   
2022-11-04 00:25:06.011213244        39     8    0.000000   True   
2022-11-04 00:25:06.011213244        39    16    0.000000   True   
2022-11-04 00:25:06.011213244        39    40    0.000000   True   
2022-11-04 00:25:06.011213244        39    33    0.000000   True   
...                                 ...   ...         ...    ...   
2022-11-04 00:29:03.921299673        12    14  -10.256484   True   
2022-11-04 00:29:03.921299673        12    34    0.000000   True   
2022-11-04 00:29:03.921299673        12    31  143.544619   True   
2022-11-04 00:29:03.921299673        12    39  149.906697   True   
2022-11-04 00:29:03.921299673        12     6  143.774841   True   

                               AppSpecificScore  IPColoca

NameError: name 'scores_min_max_scaled' is not defined